In [8]:
import numpy as np
import torch
from transformers import *
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity
import time
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [9]:
embeddings = {}
embeddings['sents'] = np.load('openaiGPT/sentences.npy')
embeddings['vectors'] = np.load('openaiGPT/vectors.npy')
embeddings['domain'] = np.load('openaiGPT/domain.npy')
embeddings['sentiment'] = np.load('openaiGPT/sentiment.npy')

In [55]:
def domain_cosine_sort(embeddings, domain='Music'):
    count = 0
    sorted_embeddings = {}
#     known_sample = {}
#     known_sample['sents'] = []
#     known_sample['vectors'] = []
#     known_sample['sentiment'] = []
    known_sample = np.zeros((embeddings['vectors'].shape[1], ))
    for i, vector in enumerate(embeddings['vectors']):
        if embeddings['domain'][i] == domain:
            known_sample += vector
            count += 1
        if count == 500:
            break
    known_sample /= 500.0
    
    count = 1
    score = []
    for i, vector in enumerate(embeddings['vectors']):
        cosine_score = cosine_similarity(vector.reshape(1,-1), known_sample.reshape(1,-1))
        score.append(cosine_score)
    score = np.array(score)
    sort_index = np.argsort(score)
    sorted_score = [score[i] for i in sort_index]
    print(sorted_score)
    sorted_embeddings['sents'] = [embeddings['sents'][i] for i in sort_index]
    sorted_embeddings['vectors'] = [embeddings['vectors'][i] for i in sort_index]
    sorted_embeddings['domain'] = [embeddings['domain'][i] for i in sort_index]
    sorted_embeddings['sentiment'] = [embeddings['sentiment'][i] for i in sort_index]
    return sorted_embeddings

def sampling(sorted_embeddings, neg_sampling=False):
    dataset = {}
    dataset['sents'] = []
    dataset['vectors'] = []
    dataset['domain'] = []
    dataset['sentiment'] = []
    dataset['labels'] = []
    for i in range(5000):
            dataset['sents'].append(sorted_embeddings['sents'][i])
            dataset['vectors'].append(sorted_embeddings['vectors'][i])
            dataset['domain'].append(sorted_embeddings['domain'][i])
            dataset['sentiment'].append(sorted_embeddings['sentiment'][i])
            dataset['labels'].append(1)
    
    if not neg_sampling:
        random_index = np.arange(5000, len(sorted_embeddings['sents']), 1)
        np.random.shuffle(random_index)
        random_index = random_index[:5000]
        for idx in random_index:
            dataset['sents'].append(sorted_embeddings['sents'][idx])
            dataset['vectors'].append(sorted_embeddings['vectors'][idx])
            dataset['domain'].append(sorted_embeddings['domain'][idx])
            dataset['sentiment'].append(sorted_embeddings['sentiment'][idx])
            dataset['labels'].append(0)
    
    else:
        for i in range(1, 5001):
            idx = -1 * i
            dataset['sents'].append(sorted_embeddings['sents'][idx])
            dataset['vectors'].append(sorted_embeddings['vectors'][idx])
            dataset['domain'].append(sorted_embeddings['domain'][idx])
            dataset['sentiment'].append(sorted_embeddings['sentiment'][idx])
            dataset['labels'].append(0)
    return dataset

In [56]:
def domain_P_R_F(dataset, domain):
    tp = sum([1 for i in range(len(dataset['labels'])) if (dataset['labels'][i] == 1 and dataset['domain'][i] == domain)])
    fp = sum([1 for i in range(len(dataset['labels'])) if (dataset['labels'][i] == 1 and dataset['domain'][i] != domain)])
    fn = sum([1 for i in range(len(dataset['labels'])) if (dataset['labels'][i] == 0 and dataset['domain'][i] == domain)])
    print(tp, fp, fn)
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    F1 = (2 * precision * recall) / (precision + recall)
    return precision, recall, F1

DOMAINS = ['Automotive', 'Books', 'Music', 'Software', 'Baby']
for domain in DOMAINS:
    sorted_embeddings = domain_cosine_sort(embeddings, domain=domain)
    dataset = sampling(sorted_embeddings, neg_sampling=False)
    print(len(dataset['labels']))
    precision, recall, F1 = domain_P_R_F(dataset, domain)
    print(f'Domain: {domain} => Precision: {precision} | Recall: {recall} | F1: {F1}')

[array([[[[0.72907773]]]]), array([[[[0.72907773]]]]), array([[[[0.72907773]]]]), array([[[[0.72907773]]]]), array([[[[0.72907773]]]]), array([[[[0.72907773]]]]), array([[[[0.72907773]]]]), array([[[[0.72907773]]]]), array([[[[0.72907773]]]]), array([[[[0.72907773]]]]), array([[[[0.72907773]]]]), array([[[[0.72907773]]]]), array([[[[0.72907773]]]]), array([[[[0.72907773]]]]), array([[[[0.72907773]]]]), array([[[[0.72907773]]]]), array([[[[0.72907773]]]]), array([[[[0.72907773]]]]), array([[[[0.72907773]]]]), array([[[[0.72907773]]]]), array([[[[0.72907773]]]]), array([[[[0.72907773]]]]), array([[[[0.72907773]]]]), array([[[[0.72907773]]]]), array([[[[0.72907773]]]]), array([[[[0.72907773]]]]), array([[[[0.72907773]]]]), array([[[[0.72907773]]]]), array([[[[0.72907773]]]]), array([[[[0.72907773]]]]), array([[[[0.72907773]]]]), array([[[[0.72907773]]]]), array([[[[0.72907773]]]]), array([[[[0.72907773]]]]), array([[[[0.72907773]]]]), array([[[[0.72907773]]]]), array([[[[0.72907773]]]]), 

10000
0 5000 0


ZeroDivisionError: division by zero

In [27]:
sorted_embeddings = domain_cosine_sort(embeddings, domain=domain)
dataset = sampling(sorted_embeddings, neg_sampling=False)

In [47]:
tp = sum([1 for i in range(len(dataset['labels'])) if (dataset['labels'][i] == 1 and dataset['domain'][i] == 'Music')])
print(tp)

5000
